# Implementation of a Devito skew self adjoint variable density visco- acoustic isotropic modeling operator <br>-- Nonlinear Ops --

## This operator is contributed by Chevron Energy Technology Company (2020)

This operator is based on simplfications of the systems presented in:
<br>**Self-adjoint, energy-conserving second-order pseudoacoustic systems for VTI and TTI media for reverse time migration and full-waveform inversion** (2016)
<br>Kenneth Bube, John Washbourne, Raymond Ergas, and Tamas Nemeth
<br>SEG Technical Program Expanded Abstracts
<br>https://library.seg.org/doi/10.1190/segam2016-13878451.1

## Introduction 

The goal of this tutorial set is to generate and prove correctness of modeling and inversion capability in Devito for variable density visco- acoustics using an energy conserving form of the wave equation. We describe how the linearization of the energy conserving *skew self adjoint* system with respect to modeling parameters allows using the same modeling system for all nonlinear and linearized forward and adjoint finite difference evolutions. 

This notebook implements the nonlinear modeling operations and is the first in a set of 3, continuing with a notebook that implements the linearization and Jacobian ops ([ssa_02_iso_implementation2.ipynb](ssa_02_iso_implementation2.ipynb)), and finally a notebook that tests the correctness of the implemented operators ([ssa_03_iso_correctness.ipynb](ssa_03_iso_correctness.ipynb)). Note there are similar notebooks for VTI and TTI anisotropy modeling operators ([README.md](README.md)).

Below we show the *skew self adjoint* form of the variable density scalar acoustic wave equation with a simple form of dissipation only Q attenuation that allows unifying modeling physics in the interior and the absorbing boundary region. This dissipation only (no dispersion) attenuation term $\left (\frac{\displaystyle \omega}{Q} \partial_t P \right)$ is an approximation of a [Maxwell Body](https://en.wikipedia.org/wiki/Maxwell_material) -- that is to say viscoelasticity approximated with a spring and dashpot in series. In practice this approach for attentuating outgoing waves is very similar to the Cernan style damping in absorbing boundaries used elsewhere in Devito ([[References]](#nl_refs)). 

The derivation of the attenuation model is not in scope for this tutorial, but one important point is that the physics in the absorbing boundary region and the interior of the model are *unified*, allowing the same modeling equations to be used everwhere, with physical Q values in the interior tapering to small Q at the boundaries to attentuate outgoing waves. 

## Outline 
1. Define symbols [[link]](#nl_symbols) 
2. Introduce the SSA wave equation [[link]](#nl_we) 
3. Show generation of skew symmetric derivatives and prove correctness with unit test [[link]](#nl_ssa_derivs) 
4. Derive the time update equation used to implement the nonlinear forward modeling operator [[link]](#nl_time_update) 
5. Create the Devito grid and model fields [[link]](#nl_grid_and_model) 
6. Define a function to implement the attenuation profile ($\omega\ /\ Q$) [[link]](#nl_attenuation) 
7. Create the Devito operator [[link]](#nl_build_operator) 
8. Run the Devito operator [[link]](#nl_run_operator) 
9. Plot the resulting wavefields [[link]](#nl_plot_wavefield) 
10. References [[link]](#nl_refs) 

<a id="nl_symbols"></a>
## Table of symbols

| Symbol &nbsp; &nbsp; &nbsp; &nbsp; | Description  | Dimensionality | 
| :---         | :---              | :--- |
| $\omega_c = 2 \pi f_c$ | center angular frequency | constant |
| $v(x,y,z)$   | P wave velocity    | function of space |
| $b(x,y,z)$   | buoyancy $(1 / \rho)$ | function of space |
| $Q(x,y,z)$   | Attenuation at frequency $\omega_c$ | function of space |
| $P(t,x,y,z)$ | Pressure wavefield | function of time and space |
| $s(t,x,y,z)$ | Source term        | function of time, localized in space |
| $\partial_t, \partial_{tt}$ | first and second derivatives wrt $t$ | time |
| $\overrightarrow{\partial_x},\overrightarrow{\partial_y},\overrightarrow{\partial_z}$ | + shifted first derivative wrt $x,y,z$ | shifted 1/2 cell forward in space |
| $\overleftarrow{\partial_x},\overleftarrow{\partial_y},\overleftarrow{\partial_z}$ | - shifted first derivative wrt $x,y,z$ | shifted 1/2 cell backward in space |

## A word about notation 

We use the arrow symbols over derivatives $\overrightarrow{\partial_x}$ as a shorthand notation to indicate that the derivative is taken at a shifted location. For example:

- $\overrightarrow{\partial_x} P$ indicates that the $x$ derivative of $P(t,x,y,z)$ is taken at $P(t,x+\frac{\Delta x}{2},y,z)$.

- $\overleftarrow{\partial_z} P$ indicates that the $z$ derivative of $P(t,x,y,z)$ is taken at $P(t,x,y,z-\frac{\Delta z}{2})$.

- $\overleftarrow{\partial_t} P$ indicates that the $t$ derivative of $P(t,x,y,z)$ is taken at $P(t-\frac{\Delta t}{2},x,y,z)$.

<a id="nl_we"></a>
## SSA variable density visco- acoustic wave equation

Our skew self adjoint wave equation is written:

$$
\frac{b}{v^2} \left( \frac{\omega_c}{Q} \overleftarrow{\partial_t} P + \partial_{tt} P \right) =
    \overleftarrow{\partial_x}\left(b\ \overrightarrow{\partial_x} P \right) +
    \overleftarrow{\partial_y}\left(b\ \overrightarrow{\partial_y} P \right) +
    \overleftarrow{\partial_z}\left(b\ \overrightarrow{\partial_z} P \right) + s 
$$

An advantage of this form is that the same system can be used to provide stable modes of propagation for all operations needed in quasi- Newton optimization: 

- the nonlinear forward 
- the linearized forward (Jacobian forward)
- the linearized adjoint (Jacobian adjoint)

This advantage is more important for anisotropic operators, where widely utilized non energy conserving formulations can provide unstable adjoints and thus unstable gradients for anisotropy parameters.  

The *skew self adjoint* formulation is evident in the shifted spatial derivatves, with the derivative on the right side $\overrightarrow{\partial}$ shifting forward in space one-half cell, and the derivative on the left side $\overleftarrow{\partial}$ shifting backward in space one-half cell. $\overrightarrow{\partial}$ and $\overleftarrow{\partial}$ are anti-symmetric (also known as skew symmetric), meaning that for two random vectors $x_1$ and $x_2$:

$$
x_2 \cdot \left( \overrightarrow{\partial_x}\ x_1 \right) \approx -\ 
x_1 \cdot \left( \overleftarrow{\partial_x}\ x_2 \right) 
$$

Below we will demonstrate this skew symmetry with a simple unit test on Devito generated shifted first derivatives. 

In the following notebooks in this series, material parameters *sandwiched* between the derivatives -- including anisotropy parameters -- become much more interesting, but here buoyancy $b$ is the only material parameter in the right hand side of our skew self adjoint (SSA) wave equation. 

## Imports 

We have grouped all imports used in this notebook here for consistency.

In [1]:
import numpy as np
from examples.seismic import RickerSource, Receiver, TimeAxis
from devito import (Grid, Function, TimeFunction, SpaceDimension, Constant, 
                    Eq, Operator, solve, configuration)
from devito.finite_differences import Derivative
from devito.builtins import gaussian_smooth
import matplotlib.pyplot as plt
from matplotlib import cm
from timeit import default_timer as timer

# These lines force images to be displayed in the notebook, and scale up fonts 
%matplotlib inline
mpl.rc('font', size=14)

# We define 32 bit floating point as the precision type 
dtype = np.float32

# Set the default language to openmp
configuration['language'] = 'openmp'

# Set logging to debug, captures statistics on the performance of operators
configuration['log-level'] = 'DEBUG'

NameError: name 'mpl' is not defined

<a id="nl_ssa_derivs"></a>
## Unit test demonstrating skew symmetry for shifted derivatives

As noted above, we demonstrate with a small 1D unit test and 8th order spatial operator that the Devito shifted first derivatives are skew symmetric. This anti-symmetry can demonstrated for the forward and backward half cell shift first derivative operators $\overrightarrow{\partial}$ and $\overleftarrow{\partial}$ with two random vectors $x_1$ and $x_2$ by verifying the *dot product test* also written above.

We will use Devito to implement the following two equations with an ```Operator```:

$$
\begin{aligned}
f_2 = \overrightarrow{\partial_x}\ f_1 \\[5pt]
g_2 = \overleftarrow{\partial_x}\ g_1
\end{aligned}
$$

And then verify the dot products are equivalent:  

$$
f_1 \cdot g_2 \approx g_1 \cdot f_2
$$

We use the following test for relative error:

$$
\frac{\displaystyle f_1 \cdot g_2 - g_1 \cdot f_2}
{\displaystyle f_1 \cdot g_2 + g_1 \cdot f_2} < \epsilon
$$


In [ ]:
# Make 1D grid to test derivatives 
n_1d = 101
d_1d = 1.0
shape_1d = (n_1d,)
spacing_1d = (1/(n_1d-1),)   
origin_1d = (0.,)
extent_1d = (d_1d*(n_1d-1),)

# Initialize Devito grid and Functions for input(f1,g1) and output(f2,g2)
grid_1d = Grid(shape=shape_1d, extent=extent_1d, origin=origin_1d, dtype=dtype)
x = grid_1d.dimensions[0]
f1 = Function(name='f1', grid=grid_1d, space_order=8)
f2 = Function(name='f2', grid=grid_1d, space_order=8)
g1 = Function(name='g1', grid=grid_1d, space_order=8)
g2 = Function(name='g2', grid=grid_1d, space_order=8)

# Fill f1 and g1 with random values in [-1,+1]
f1.data[:] = -1 + 2 * np.random.rand(n_1d,)
g1.data[:] = -1 + 2 * np.random.rand(n_1d,)

# Equation defining: [f2 = forward 1/2 cell shift derivative applied to f1]
equation_f2 = Eq(f2, getattr(f1,'dx')(x+0.5*x.spacing))

# Equation defining: [g2 = backward 1/2 cell shift derivative applied to g1]
equation_g2 = Eq(g2, getattr(g1,'dx')(x-0.5*x.spacing))

# Define an Operator to implement these equations and execute
op = Operator([equation_f2, equation_g2])
op()

# Compute the dot products and the relative error
f1g2 = + np.dot(f1.data, g2.data)
g1f2 = - np.dot(g1.data, f2.data)   # skew symmetry requires this minus sign
diff = (f1g2-g1f2)/(f1g2+g1f2)

tol = 100 * np.finfo(np.float32).eps
print("f1g2, g1f2, diff, tol; %+.6e %+.6e %+.6e %+.6e" % (f1g2, g1f2, diff, tol))

# At last the unit test
# Assert these dot products are float epsilon close in relative error
assert diff < 100 * np.finfo(np.float32).eps

## Show the finite difference operators and generated code

You can inspect the finite difference coefficients and locations for evaluation with the code shown below. 

Note that you don't need to inspect the generated code, but this does provide the option to use this highly optimized code in applications that do not need or require python.

In [ ]:
# Show the FD coefficients generated by Devito 
# for the forward 1/2 cell shifted first derivative operator
print("\n\nForward +1/2 cell shift;")
print("..................................")
print(f1.dx(x0=x+0.5*x.spacing).evaluate)

# Show the FD coefficients generated by Devito 
# for the backward 1/2 cell shifted first derivative operator
print("\n\nBackward -1/2 cell shift;")
print("..................................")
print(f1.dx(x0=x-0.5*x.spacing).evaluate)

# Show code generated by Devito for applying the derivatives
print("\n\nGenerated c++ code;")
print("..................................")
print(op.ccode)

<a id="nl_time_update"></a>
## Doing some algebra to solve for the time update

The next step in implementing our Devito modeling operator is to define the equation used to update the pressure wavefield as a function of time. What follows is a bit of algebra using the wave equation and finite difference approximations to time derivatives to express the pressure wavefield forward in time $P(t+\delta t)$ as a function of the current $P(t)$ and previous $P(t-\delta t)$ pressure wavefields.

<br>

#### 1. Numerical approximation for $\partial_{tt}P$
The second order accurate centered approximation to the second time derivative involves three wavefields: $P(t-\delta t)$, $P(t)$, and $P(t+\delta t)$. 

<br>
$$
\partial_{tt} P = \frac{\displaystyle P(t+\delta t) - 2 P(t) + P(t-\delta t)}{\displaystyle \delta t^2} 
$$

<br>

#### 2. Solve equation from step (1) for $P(t+\delta t)$
In order to advance our finite difference solution in time, we solve for $P(t+\delta t)$.

<br>
$$
P(t+\delta t) = \delta t^2 \partial_{tt} P 
    + 2 P(t) - P(t-\delta t)
$$

<br>

#### 3. Numerical approximation for $\overleftarrow{\partial_{t}}P$
The argument for using a backward approximation is a bit hand wavy, but goes like this: a centered or forward approximation for $\partial_{t}P$ would involve the term $P(t+\delta t)$, and hence $P(t+\delta t)$ would appear at two places in our time update equation below, essentially making the form implicit. 

We are interested in explicit time stepping and so prefer the backward approximation for $\overleftarrow{\partial_{t}}P$.  Our experience is that the use of the backward difference is more stable than forward or centered.

The first order accurate backward approximation to the first time derivative involves two wavefields: $P(t-\delta t)$, and $P(t)$. We can use this expression as is. 

<br>
$$
\overleftarrow{\partial_{t}}P = \frac{\displaystyle P(t) - P(t-\delta t)}{\displaystyle \delta t}
$$

<br>

#### 4. Solve the wave equation for $\partial_{tt}P$

<br>
$$
\begin{aligned}
\frac{b}{v^2} \left( \frac{\omega_c}{Q} \overleftarrow{\partial_{t}}P +
    \partial_{tt}P \right) &=
    \overleftarrow{\partial_x}\left(b\ \overrightarrow{\partial_x} P \right) +
    \overleftarrow{\partial_y}\left(b\ \overrightarrow{\partial_y} P \right) +
    \overleftarrow{\partial_z}\left(b\ \overrightarrow{\partial_z} P \right) + s\\[10pt]
    \partial_{tt}P &=
    \frac{v^2}{b} \left[ 
        \overleftarrow{\partial_x}\left(b\ \overrightarrow{\partial_x} P \right) +
        \overleftarrow{\partial_y}\left(b\ \overrightarrow{\partial_y} P \right) +
        \overleftarrow{\partial_z}\left(b\ \overrightarrow{\partial_z} P \right) + s
    \right] 
    - \frac{\omega_c}{Q} \overleftarrow{\partial_{t}}P
\end{aligned}
$$

<br>

#### 5. Plug equations from steps (3) and (4) into equation from step (2)
Next we plug in the expression for $\partial_{tt}P$ and $\partial_{t}P$ into the the time update expression for $P(t+\delta t)$ from step 2.

<br>
$$
\begin{aligned}
    P(t+\delta t) &= \delta t^2 
        \frac{v^2}{b} \left[ 
            \overleftarrow{\partial_x}\left(b\ \overrightarrow{\partial_x} P \right) +
            \overleftarrow{\partial_y}\left(b\ \overrightarrow{\partial_y} P \right) +
            \overleftarrow{\partial_z}\left(b\ \overrightarrow{\partial_z} P \right) + s
        \right] \\[10pt]
        & \quad -\ \delta t^2 \frac{\omega_c}{Q}
            \left( \frac{\displaystyle P(t) - P(t-\delta t)}{\displaystyle \delta t} \right) 
            + 2 P(t) - P(t-\delta t)
\end{aligned}
$$

<br>

#### 6. Simplify ...

<br>
$$
\begin{aligned}
    P(t+\delta t) &=
        \frac{\delta t^2 v^2}{b} \left[ 
            \overleftarrow{\partial_x}\left(b\ \overrightarrow{\partial_x} P \right) +
            \overleftarrow{\partial_y}\left(b\ \overrightarrow{\partial_y} P \right) +
            \overleftarrow{\partial_z}\left(b\ \overrightarrow{\partial_z} P \right) + s
        \right] \\[10pt]
        &\quad -\ \frac{\delta t\ \omega_c}{Q} \Bigr[P(t) - P(t-\delta t)\Bigr] 
        + 2 P(t) - P(t-\delta t)
\end{aligned}
$$

<br>

#### compare to c++ code
```pOld[k] = dt2V2_B * (dPx + dPy + dPz) - dtOmegaInvQ[k] * (pCur[k] - pOld[k]) - pOld[k] + 2 * pCur[k];```

<br>

#### 7. et voila ...
The last equation is how we update the pressure wavefield at each time step, and depends on $P(t)$ and $P(t-\delta t)$. 

The main work of the finite difference explicit time stepping is evaluating the nested spatial derivative operators on the RHS of this equation. The particular advantage of Devito symbolic optimization is that Devito is able to solve for the complicated expressions that result from substituting the discrete forms of high order numerical finite difference approximations for these nested spatial derivatives.

We have now completed the maths required to implement the modeling operator. The remainder of this notebook deals with setting up and using the required Devito objects.


<a id="nl_grid_and_model"></a>
## Instantiate the Devito grid for a two dimensional problem

Define the dimensions and coordinates for the model. The computational domain of the model is surrounded by an *absorbing boundary region* where we implement boundary conditions to eliminate outgoing waves. We define the sizes for the interior of the model ```nx``` and ```nz```, the width of the absorbing boundary region ```npad```, and the sizes for the entire model padded with absorbing boundaries become ```nxpad = nx + 2*npad``` and ```nzpad = nz + 2*npad```.

In [ ]:
# Define dimensions for the interior of the model
nx,nz = 751,751
dx,dz = 10.0,10.0                # Grid spacing in m
shape = (nx, nz)                 # Number of grid points
spacing = (dx, dz)               # Domain size is now 5 km by 5 km
origin = (0., 0.)                # Origin of coordinate system, specified in m.
extent = tuple([s*(n-1) for s, n in zip(spacing, shape)])

# Define dimensions for the model padded with absorbing boundaries
npad        = 50                 # number of points in absorbing boundary region (all sides)
nxpad,nzpad = nx+2*npad, nz+2*npad
shape_pad   = np.array(shape) + 2 * npad
origin_pad  = tuple([o - s*npad for o, s in zip(origin, spacing)])
extent_pad  = tuple([s*(n-1) for s, n in zip(spacing, shape_pad)])

# Initialize the Devito grid 
x = SpaceDimension(name='x', spacing=Constant(name='h_x', value=extent_pad[0]/(shape_pad[0]-1)))
z = SpaceDimension(name='z', spacing=Constant(name='h_z', value=extent_pad[1]/(shape_pad[1]-1)))
grid = Grid(extent=extent_pad, shape=shape_pad, origin=origin_pad, dimensions=(x, z), dtype=dtype)

print("shape;           ", shape)
print("origin;          ", origin)
print("spacing;         ", spacing)
print("extent;          ", extent)
print("")
print("shape_pad;       ", shape_pad)
print("origin_pad;      ", origin_pad)
print("extent_pad;      ", extent_pad)

print("")
print("grid.shape;      ", grid.shape)
print("grid.extent;     ", grid.extent)
print("grid.spacing_map;", grid.spacing_map)

## Define velocity and buoyancy model parameters

We have the following constants and fields from our SSA wave equation that we define as time invariant using ```Functions```:

| &nbsp; Symbol &nbsp; | Description |
| :---: | :--- |
| $v(x,z)$ | Acoustic velocity |
| $$b(x,z)=\frac{1}{\rho(x,z)}$$ | Buoyancy (reciprocal density) |

In [ ]:
# Create the velocity and buoyancy fields. 
#   - We use a wholespace 1500 m/s velocity model here
#   - These are scalar fields so we use Function to define them
#   - We specify space_order to establish the appropriate size halo on the edges 
space_order = 8

# Wholespace velocity
v = Function(name='v', grid=grid, space_order=space_order)
v.data[:] = 1.5

# Constant density
b = Function(name='b', grid=grid, space_order=space_order)
b.data[:,:] = 1.0 / 1000.0

## Define the simulation time range

In this notebook we run 2 seconds of simulation using the sample rate related to the CFL condition as implemented in ```examples/seismic/model.py```. 

**Important note** smaller Q values in highly viscous media may require smaller temporal sampling rates than a non-viscous medium to achieve dispersion free propagation. This is a cost of the visco- acoustic modification we use here.

We also use the convenience ```TimeRange``` as defined in ```examples/seismic/source.py```.

In [ ]:
t0 = dtype(0.)     # Simulation time start
tn = dtype(2000.)  # Simulation time end (1 second = 1000 msec)
dt = np.float32(0.38 * np.min(spacing) / np.max(v.data))
# dt = np.float32(0.19 * np.min(spacing) / np.max(v.data))
# dt = 2.0
# dt = 1.0
# dt = 0.5
time_range = TimeAxis(start=t0, stop=tn, step=dt)
print("Time min, max, dt, num; %10.6f %10.6f %10.6f %d" % (t0, tn, dt, int(tn//dt) + 1))
print("time_range; ", time_range)

## Define the acquisition geometry: locations of sources and receivers 

**source**:
- X coordinate: center of the model: dx*(nx//2)
- Z coordinate: in the middle of the upper layer: dz*(nz//4)
- We use a 10 Hz center frequency [RickerSource](https://github.com/devitocodes/devito/blob/master/examples/seismic/source.py#L280) wavelet source as defined in ```examples/seismic/source.py```

**receivers**:
- X coordinate: center of the model: dx*(nx//2)  
- Z coordinate: vertical line from top to bottom of model
- We use a [Receiver](https://github.com/devitocodes/devito/blob/master/examples/seismic/source.py#L80) as defined with a ```PointSource``` in ```examples/seismic/source.py```

In [ ]:
# Source in the center of the model at 10 Hz center frequency
fpeak = 0.010
src = RickerSource(name='src', grid=grid, f0=fpeak, npoint=1, time_range=time_range)
src.coordinates.data[0,0] = dx * (nx//2)
src.coordinates.data[0,1] = dz * (nz//2)

# line of receivers along the right edge of the model
rec = Receiver(name='rec', grid=grid, npoint=nz, time_range=time_range)
rec.coordinates.data[:,0] = dx * (nx//2)
rec.coordinates.data[:,1] = np.linspace(0.0, dz*(nz-1), nz)

print("src_coordinate  X;         %+12.4f" % (src.coordinates.data[0,0]))
print("src_coordinate  Z;         %+12.4f" % (src.coordinates.data[0,1]))
print("src_coordinates X min/max; %+12.4f %+12.4f" % \
      (np.min(rec.coordinates.data[:,0]), np.max(rec.coordinates.data[:,0])))
print("src_coordinates Z min/max; %+12.4f %+12.4f" % \
      (np.min(rec.coordinates.data[:,1]), np.max(rec.coordinates.data[:,1])))

# We can plot the time signature to see the wavelet
src.show()

## Plot velocity model

Next we plot the half-space velocity model for illustration. 
- The demarcation between interior and absorbing boundary is shown with a dotted white line
- The source is shown as a large red asterisk
- The extent of the receiver array is shown with a thick dashed blue line

In [ ]:
# note: flip sense of second dimension to make the plot positive downwards
plt_extent = [origin_pad[0], origin_pad[0] + spacing[0]*(shape_pad[0]-1), 
              origin_pad[1] + spacing[1]*(shape_pad[1]-1), origin_pad[1]]

plt.figure(1, figsize=(12,8))
plt.clf()
plt.imshow(np.transpose(v.data), cmap=cm.jet, vmin=1.4, vmax=1.7, extent=plt_extent)
plt.plot([origin[0], origin[0], extent[0], extent[0], origin[0]], 
         [origin[1], extent[1], extent[1], origin[1], origin[1]], 
         'black', linewidth=4, linestyle=':', label="Absorbing Boundary")
plt.plot(rec.coordinates.data[:, 0], rec.coordinates.data[:, 1], \
         'blue', linewidth=5, linestyle='--', label="Receiver line")
plt.plot(src.coordinates.data[:, 0], src.coordinates.data[:, 1], \
         'red', linestyle='None', marker='*', markersize=15, label="Source")
ax = plt.legend(bbox_to_anchor=(1.25, 0.5), loc='upper left')
ax.get_frame().set_color("gray")
plt.colorbar()
plt.xlabel("X Coordinate (m)", labelpad=15)
plt.ylabel("Z Coordinate (m)", labelpad=15)
plt.title("Wholespace velocity model including absorbing boundary region", y=1.025)
plt.tight_layout()
None

<a id="nl_attenuation"></a>
## Create the $\omega_c\ /\ Q(x,z)$ model used for dissipation only attenuation

We have the following constants and fields from our SSA wave equation that we need to define and instantiate:

| &nbsp; Symbol &nbsp; | Description |
| :---: | :--- |
| $\omega_c = 2 \pi f_c$      | Center angular frequency |
| $$\frac{1}{Q(x,z)}$$ | Inverse Q model used in the modeling system |

The strategy we use is designed to eliminate any corners or edges in the attentuation profile. We do this by making Q as a function of *distance from the **nearest** boundary*.

## Define a python function for setting $\omega_c\ /\ Q(x,z)$ array 

The python code in the function ```setup_wOverQ``` below uses the ```numpy.ogrid``` function, which allows an *outer product* over index arrays for the two dimensions to build up the attenuation profile. This approach is about 100x faster than using nested loops in python. This function is defined in the ```devito/examples/seismic/skew_self_adjoint``` directory but shown here for completeness.

In the next three cells we will:
1. Define a function for setting up the $\omega_c\ /\ Q(x,z)$ array
2. Use that function to establish the array we need to implement attenuation as a ```Function```
3. Plot the generated Q profile 

In [ ]:
def setup_wOverQ(wOverQ, w, qmin, qmax, npad, sigma=npad//5):
    """
    Initialise spatially variable w/Q field used to implement attenuation and
    absorb outgoing waves at the edges of the model.

    Parameters
    ----------
    wOverQ : Function
        The omega over Q field used to implement attenuation in the model, 
        and the absorbing boundary condition for outgoing waves.
    w : float32
        center angular frequency, e.g. peak frequency of Ricker source wavelet 
        used for modeling.
    qmin : float32
        Q value at the edge of the model. Typically set to 0.1 to strongly 
        attenuate outgoing waves.
    qmax : float32
        Q value in the interior of the model. Values > 1000 are effectively
        no attenuation.
    npad : int
        Number of points in the absorbing boundary region. Note that we expect 
        this to be the same on all sides of the model.
    sigma : float32
        sigma value for call to scipy gaussian smoother, default 5.
    """
    assert w > 0, "supplied w value [%f] must be positive" % (w)
    assert qmin > 0, "supplied qmin value [%f] must be positive" % (qmin)
    assert qmax > 0, "supplied qmax value [%f] must be positive" % (qmax)
    assert npad > 0, "supplied npad value [%f] must be positive" % (npad)
    
    t1 = timer()
    lqmin = np.log(qmin);
    lqmax = np.log(qmax);

    kxMin,kzMin = np.ogrid[:nxpad,:nzpad]
    kxArr,kzArr = np.minimum(kxMin,nxpad-1-kxMin),np.minimum(kzMin,nzpad-1-kzMin)
    nval1 = np.minimum(kxArr,kzArr)
    nval2 = np.minimum(1,nval1/(npad))
    nval3 = np.exp(lqmin+nval2*(lqmax-lqmin))

    # Note if we apply gaussian smoother, we must renormalize to [qmin,qmax] 
    if sigma > 0:
        nval2[:] = gaussian_smooth(nval3, sigma=sigma)
        nmin2,nmax2 = np.min(nval2), np.max(nval2)
        nval3[:] = qmin + (qmax - qmin) * (nval2 - nmin2) / (nmax2 - nmin2)

    wOverQ.data[:,:] = w / nval3

    # report min/max output Q value
    q1 = (np.min(1 / (wOverQ.data / w)))
    q2 = (np.max(1 / (wOverQ.data / w)))
    t2 = timer()
    print("setup_wOverQ ran in %.4f seconds -- min/max Q values; %8.4f %8.4f" % (t2-t1, q1, q2)) 

## Initialize the attenuation profile

Note that we use a fairly strong attenuation -- here with a Q value of 10 -- in order to demonstrate the impact of the attenuation on the amplitude in the receiver gather below. 

In [ ]:
# Initialize the attenuation profile
w = 2.0 * np.pi * fpeak
print("w,fpeak; ", w, fpeak)
qmin = 0.1
qmax = 10.0
wOverQ = Function(name='wOverQ', grid=grid, space_order=space_order)
setup_wOverQ(wOverQ, w, qmin, qmax, npad)
# wOverQ.data[:] = 0.0

In [ ]:
# Plot the generated Q profile
Q = np.log10(1 / (wOverQ.data / w)) 
plt.figure(1, figsize=(12,8))
plt.clf()
plt.imshow(np.transpose(Q), cmap=cm.jet_r, vmin=np.log10(qmin), vmax=np.log10(qmax), extent=plt_extent)
plt.plot([origin[0], origin[0], extent[0], extent[0], origin[0]], 
         [origin[1], extent[1], extent[1], origin[1], origin[1]], 
         'white', linewidth=4, linestyle=':', label="Absorbing Boundary")
ax = plt.legend(bbox_to_anchor=(1.25, 0.5), loc='upper left')
ax.get_frame().set_color("gray")
plt.xlabel("X Coordinate (m)", labelpad=15)
plt.ylabel("Z Coordinate (m)", labelpad=15)
plt.title("log10 of $Q$ value", y=1.025)
plt.colorbar()
plt.tight_layout()
None

## Define the pressure wavefield as a ```TimeFunction```

We specify the time_order as 2, which allocates 3 time steps in the pressure wavefield. As described elsewhere, Devito will use "cyclic indexing" to index into this multi-dimensional array.

In [ ]:
# Define the TimeFunction
p = TimeFunction(name="p", grid=grid, time_order=2, space_order=space_order)

# Get the dimensions for t, x, z 
t,x,z = p.dimensions

## Define the source injection and receiver extraction 

In [ ]:
# Finally we define the source injection and receiver extraction functions 
src_term = src.inject(field=p.forward, expr=src * t.spacing**2 * v**2 / b)

# Create interpolation expression for receivers
rec_term = rec.interpolate(expr=p.forward)

<a id="nl_build_operator"></a>
## Finally, the Devito operator

We next transcribe the time update expression above into a Devito ```Eq```. Then we add the source injection and receiver extraction and build an ```Operator``` that will generate the c++ code for performing the model.

We copy the time update expression from above for clarity. Note we omit $s$ from the ```Eq``` below because we will be explicitly injecting the source using ```src_term```.

$$
\begin{aligned}
    P(t+\delta t) &=
        \frac{\delta t^2 v^2}{b} \left[ 
            \overleftarrow{\partial_x}\left(b\ \overrightarrow{\partial_x} P \right) +
            \overleftarrow{\partial_y}\left(b\ \overrightarrow{\partial_y} P \right) +
            \overleftarrow{\partial_z}\left(b\ \overrightarrow{\partial_z} P \right) + s
        \right] \\[10pt]
        &\quad -\ \frac{\delta t\ \omega_c}{Q} \Bigr[P(t) - P(t-\delta t)\Bigr] 
        + 2 P(t) - P(t-\delta t)
\end{aligned}
$$

## Impact of hardwiring the grid spacing on flop count

The argument ```subs=p.grid.spacing_map``` passed to the operator substitutes values for the current grid spacing into the expressions before code generation. This reduces the number of floating point operations executed by the kernel by pre-evaluating certain coefficients, or possibly absorbing the spacing scalars into the finite difference coefficients.  

If you run the two cases of passing and not passing the ```subs=p.grid.spacing_map``` argument below by commenting/un-commenting the last two lines of the cell, you can inspect the difference in computed flop count for the operator. Note also if you inspect the generated code for the two cases, you will see extra calling parameters are required for the case without the substitution. We have compiled the flop count from the two runs into the table below.

| Passing subs argument | Flops reduction after symbolic optimization |
|:---|:---:|
| True | 300 --> 69 |
| False | 588 --> 81 |

Note the gain in performance is around 15% for this example in 2D, and would be more significant in 3D.

In [ ]:
# NBVAL_IGNORE_OUTPUT
eq_time_update = (t.spacing**2 * v**2 / b) * \
    ((b * p.dx(x0=x+x.spacing/2)).dx(x0=x-x.spacing/2) + \
     (b * p.dz(x0=z+z.spacing/2)).dz(x0=z-z.spacing/2)) - \
    t.spacing * wOverQ * (p - p.backward) + 2 * p - p.backward

stencil = Eq(p.forward, eq_time_update)

# op = Operator([stencil] + src_term + rec_term)
op = Operator([stencil] + src_term + rec_term, subs=p.grid.spacing_map)

print(p.grid.spacing_map)

In [ ]:
# NBVAL_IGNORE_OUTPUT
# Print the arguments used in the signature for the the c++ generated code
# op.arguments()

In [ ]:
# NBVAL_IGNORE_OUTPUT
# Print the generated c++ code
print(op)

<a id="nl_run_operator"></a>
## Run the operator

In [ ]:
# run the operator
print("v        min/max; %+12.6e %+12.6e" % (np.min(v.data), np.max(v.data)))
print("b        min/max; %+12.6e %+12.6e" % (np.min(b.data), np.max(b.data)))
print("wOverQ   min/max; %+12.6e %+12.6e" % (np.min(wOverQ.data), np.max(wOverQ.data)))
print(time_range)
summary = op(time=time_range.num-1, dt=dt)
# summary = op(time=time_range.num-1, h_x=dx, h_z=dz, dt=dt)
print("data min/max; %+12.6e %+12.6e" % (np.min(p.data[1,:,:]), np.max(p.data[1,:,:])))

<a id="nl_plot_wavefield"></a>
## Plot the computed wavefield

In [ ]:
# Plot the wavefield in the model
plt.figure(1, figsize=(12,8))
plt.clf()
amax = 1.0 * np.max(np.abs(p.data[1,:,:]))
plt.imshow(np.transpose(p.data[1,:,:]), vmin=-amax, vmax=+amax, cmap="seismic", extent=plt_extent)
plt.plot([origin[0], origin[0], extent[0], extent[0], origin[0]], 
         [origin[1], extent[1], extent[1], origin[1], origin[1]], 
         'black', linewidth=4, linestyle=':', label="Absorbing Boundary")
plt.plot(rec.coordinates.data[:, 0], rec.coordinates.data[:, 1], \
         'blue', linewidth=5, linestyle='--', label="Receiver line")
plt.plot(src.coordinates.data[:, 0], src.coordinates.data[:, 1], \
         'red', linestyle='None', marker='*', markersize=15, label="Source")
plt.colorbar()
ax = plt.legend(bbox_to_anchor=(1.25, 0.5), loc='upper left')
ax.get_frame().set_color("gray")
plt.xlabel('X coordinate (m)')
plt.ylabel('Z coordinate (m)')
plt.title('Model wavefield at t=%.0f msec for Q=%.0f' % (time_range.stop, qmax))
plt.tight_layout()

# Plot the receiver wavefield
plt.figure(figsize=(8,8))
amax = 0.1 * np.max(np.abs(rec.data[:]))
plt.imshow(rec.data[:,:], vmin=-amax, vmax=+amax, cmap="seismic", aspect="auto")
plt.colorbar()
plt.xlabel('Z coordinate (m)')
plt.ylabel('Time (msec)')
plt.title('Receiver wavefield for Q=%.0f' % (qmax))
plt.tight_layout()

## Show the output from Devito solving for the stencil

Note this takes a **long time** ... about 50 seconds, but obviates the need to solve for the time update expression as we did above.

In [ ]:
# Define the partial_differential equation
# Note the backward shifted time derivative is obtained via p.dt(x0=t-0.5*t.spacing) 
pde = (b / v**2) * (wOverQ * p.dt(x0=t-0.5*t.spacing) + p.dt2) -\
        (b * p.dx(x0=x+0.5*x.spacing)).dx(x0=x-0.5*x.spacing) -\
        (b * p.dz(x0=z+0.5*z.spacing)).dz(x0=z-0.5*z.spacing)

# t1 = timer()
# stencil = Eq(p.forward, solve(pde, p.forward))
# t2 = timer()
# print("solve ran in %.4f seconds." % (t2-t1)) 
# stencil

In [ ]:
# Define the partial_differential equation
#   Note: using @rnelson's trick of redefing the spatial and temporal derivs as 
#   sympy symbols just for the solve in order to save time, and then replacing 
#   with the actual derivatives later.
# import sympy as sp
# dxbdx = sp.Symbol('dxbdx')
# dzbdz = sp.Symbol('dzbdz')
pde = (b / v**2) * (wOverQ * p.dt(x0=t-0.5*t.spacing) + p.dt2) 
# pde = (b / v**2) * (wOverQ * p.dt(x0=t-0.5*t.spacing) + p.dt2) - dxbdx - dzbdz 
t1 = timer()
stencil = Eq(p.forward, solve(pde, p.forward))
# t2 = timer()
# print("solve ran in %.4f seconds." % (t2-t1)) 
# stencil

In [ ]:
# op = Operator([stencil] + src_term + rec_term)
op = Operator([stencil] + src_term + rec_term, subs=p.grid.spacing_map)
op()

print(op)

<a id="nl_refs"></a>
## References

- **A nonreflecting boundary condition for discrete acoustic and elastic wave equations** (1985)
<br>Charles Cerjan, Dan Kosloft. Ronnie Kosloff, and Moshe Resheq
<br> Geophysics, Vol. 50, No. 4
<br>https://library.seg.org/doi/pdfplus/10.1190/segam2016-13878451.1

- **Generation of Finite Difference Formulas on Arbitrarily Spaced Grids** (1988)
<br>Bengt Fornberg
<br>Mathematics of Computation, Vol. 51, No. 184
<br>http://dx.doi.org/10.1090/S0025-5718-1988-0935077-0
<br>https://web.njit.edu/~jiang/math712/fornberg.pdf

- **Self-adjoint, energy-conserving second-order pseudoacoustic systems for VTI and TTI media for reverse time migration and full-waveform inversion** (2016)
<br>Kenneth Bube, John Washbourne, Raymond Ergas, and Tamas Nemeth
<br>SEG Technical Program Expanded Abstracts
<br>https://library.seg.org/doi/10.1190/segam2016-13878451.1
